# <div align="center">TVLA<div>



In [ ]:
# if not already installed
# !pip install -U -e ../python_lib

In [1]:
# import libraries
import os
import sys
import shutil
import json
import numpy as np
from pathlib import Path
from dataclasses import asdict

from foboslib.projmgr import ProjectManager
from foboslib.ctrl import pynqctrl
from foboslib.ctrl.fobosctrl import FOBOSCtrl
from foboslib.tvlaCalc import TVLACalc
from foboslib.dut.cw305dut import CW305DUT
# from foboslib.dut.nexys3dut import Nexys3DUT

#######################################################
# acquisition configuration
# connection to ctrl board
acqConf = pynqctrl.Config()
# Connection to Control Board
acqConf.ip = 'localhost'
acqConf.port = 9995
# File names and locations
acqConf.workspace       = "fobosworkspace"    # Folder to store projects
acqConf.projectName     = "xoodyak-dom-1st"   # Name of this project. Files will be in workspace/projectName/
acqConf.dinFile         = "dinFile.txt"       # test vectors in FOBOS format incl. plaintext and key, auto-generated
acqConf.fvrFile         = "fvrchoicefile.txt" #
acqConf.cipherFile      = "doutFile"          # observed output, result from applying test vectors
acqConf.traceFile       = "powerTraces.npy"   # Name of file for recording power traces
# Cipher Specific Information
acqConf.bitFile         = "/nhome/mmohajer/notebooks/fobos/software/notebooks/sca_labs/fobosworkspace/aes_tvla/half_duplex_dut.bit"     # Bitstream for programming the DUT, same directory as this notebook
acqConf.outLen          = 88                  # size of a block of output in bytes
# Acquistion configuration
acqConf.traceNum        = 100                 # number of traces to run
acqConf.samplingFreq    = 50                  # sampling frequency of the Oscilloscope in Msps [range: 1 - 100]
acqConf.DUTClk          = 10                  # clock frequency of the DUT in MHz [range: 1 - 100]
acqConf.samplesPerTrace = 5*80                # number of ADC samples in one trace
acqConf.ADCGain         = 40                  # amplification of ADC input signal [range: 0 - 60]
acqConf.ADCHiLo         = 1                   # 0 - low amplification, 1 - high amplification

In [2]:
# Configure project directories
pm = ProjectManager()
pm.setWorkSpaceDir(acqConf.workspace)
pm.setProjName(acqConf.projectName)
projDir = pm.getProjDir()
print(f"projDir={projDir}")

projDir=fobosworkspace/xoodyak-dom-1st


In [7]:
# program DUT ##########################################################
# dut = Nexys3DUT()
import chipwhisperer as cw
from chipwhisperer.capture.targets.CW305 import CW305
# scope = cw.scope()
scope = None

# scope can also be None here, unlike with the default SimpleSerial
# bitFile = os.path.join(projDir, acqConf.bitFile)
bitstream = "/nhome/mmohajer/notebooks/fobos/software/notebooks/sca_labs/fobosworkspace/aes_tvla/half_duplex_dut.bit"
target = cw.target(scope, CW305, bsfile=bitstream)
target.vccint_set(1.0) #set VCC-INT to 1V
        
target.pll.pll_enable_set(True) #Enable PLL chip
target.pll.pll_outenable_set(False, 0) # Disable unused PLL0
target.pll.pll_outenable_set(True, 1)  # Enable PLL 
target.pll.pll_outenable_set(False, 2) # Disable unused PLL2

# optional, but reduces power trace noise
target.clkusbautooff = True
target.clksleeptime = 1 # 1ms typically good for sleep

# dut.setBitFile(bitFile)
# dut.program()

OSError: Could not find ChipWhisperer. Is it connected?

In [ ]:




# prepare i/o files ####################################################


tvFileName = os.path.join(projDir, acqConf.dinFile)
#plainFileName = os.path.join(projDir, acqConf.plainFile)
assert Path(tvFileName).exists()
#     with open(tvFileName, "w") as tvFile:
#         tvFile.write("")
    
captureDir = pm.getCaptureDir()

traceFileName = os.path.join(captureDir, acqConf.traceFile)
cipherFileName = os.path.join(captureDir, acqConf.cipherFile)
fvrFileName = os.path.join(projDir, acqConf.fvrFile)

shutil.copy(tvFileName, captureDir)
shutil.copy(fvrFileName, captureDir)

# save config to a file
with open(os.path.join(captureDir, 'acquisitionConfig.json'), "w") as configFile:
    configFile.write(json.dumps(asdict(acqConf), indent=4))
    # configFile.write(str(ctrl.config)) # ????!!!
    # print(str(acqConf))

# connect to hardware and lock it ######################################
tCalc = TVLACalc(acqConf.samplesPerTrace)

with pynqctrl.PYNQCtrl(acqConf) as ctrl:
    with open(tvFileName, "r") as tvFile, open(traceFileName, "a+b") as traceFile, open(cipherFileName, "w") as cipherFile, open(fvrFileName, 'r') as fvrFile:
        #t-test object
        # Get traces############################################################
        print('Processing test vectors ...')
        for traceNum in range(acqConf.traceNum):
            data = tvFile.readline()
            print(f"data={data}")
            status, result, trace = ctrl.processData2(data, acqConf.outLen)
            if result is None:
                print(f"result is None! traceNum:{traceNum} status:{status} trace:{trace}")
                break
            cipherFile.write(result + "\n")
            np.save(traceFile, trace)
            c = fvrFile.read(1)
            tCalc.addTrace(trace, int(c))
            if traceNum % 10000 == 0:
                t_array, passed = tCalc.saveData1(captureDir, str(traceNum))
                sys.stdout.write('Progress:' + "{:.2f}".format(traceNum/acqConf.traceNum*100) + '%\r')
                sys.stdout.flush()

tCalc.saveData1(captureDir, str(acqConf.traceNum)) 

print('Data acquisition complete.')



programming DUT. Please wait ...


OSError: Could not find ChipWhisperer. Is it connected?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(trace)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig.set_facecolor('white')
plt.rcParams.update({'font.size': 18})
maxtrace = 100
plt.figure(figsize=(10,8))
plt.xlabel('Sample')
plt.ylabel('T-Value')
plt.title(f'TVLA {acqConf.projectName}')

with open(traceFileName, "r+b") as traceFile:
    for i in range(min(maxtrace, acqConf.traceNum)):
        trace = np.load(traceFile, allow_pickle=True)
        plt.plot(trace)

plt.savefig(os.path.join(captureDir, 'traces.png'),facecolor=fig.get_facecolor())
# plt.close()
